In [1]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_validate, RandomizedSearchCV, validation_curve
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, validation_curve, RandomizedSearchCV

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 500)
warnings.simplefilter(action="ignore", category=Warning)

In [3]:
train = pd.read_csv("train.csv")
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
train.shape # (1460, 81)

(1460, 81)

In [5]:
train.isnull().sum().sort_values(ascending=False).head(20)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtFinType2      38
BsmtExposure      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
MasVnrType         8
Electrical         1
Id                 0
dtype: int64

In [6]:
test = pd.read_csv("test.csv")
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [7]:
test.shape # (1459, 80)

(1459, 80)

In [8]:
test.isnull().sum().sort_values(ascending=False).head(34)

PoolQC          1456
MiscFeature     1408
Alley           1352
Fence           1169
FireplaceQu      730
LotFrontage      227
GarageYrBlt       78
GarageQual        78
GarageFinish      78
GarageCond        78
GarageType        76
BsmtCond          45
BsmtQual          44
BsmtExposure      44
BsmtFinType1      42
BsmtFinType2      42
MasVnrType        16
MasVnrArea        15
MSZoning           4
BsmtHalfBath       2
Utilities          2
Functional         2
BsmtFullBath       2
BsmtFinSF1         1
BsmtFinSF2         1
BsmtUnfSF          1
KitchenQual        1
TotalBsmtSF        1
Exterior2nd        1
GarageCars         1
Exterior1st        1
GarageArea         1
SaleType           1
MiscVal            0
dtype: int64

In [9]:
df = pd.concat([train, test], axis=0)
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


In [10]:
df.shape # (2919, 81)

(2919, 81)

In [11]:
def grab_col_names(dataframe, cat_th=0, car_th=26):
    """

            Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
            Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

            Parameters
            ------
                dataframe: dataframe
                        Değişken isimleri alınmak istenilen dataframe
                cat_th: int, optional
                        numerik fakat kategorik olan değişkenler için sınıf eşik değeri
                car_th: int, optinal
                        kategorik fakat kardinal değişkenler için sınıf eşik değeri

            Returns
            ------
                cat_cols: list
                        Kategorik değişken listesi
                num_cols: list
                        Numerik değişken listesi
                cat_but_car: list
                        Kategorik görünümlü kardinal değişken listesi

            Examples
            ------
                import seaborn as sns
                df = sns.load_dataset("iris")
                print(grab_col_names(df))


            Notes
            ------
                cat_cols + num_cols + cat_but_car = toplam değişken sayısı
                num_but_cat cat_cols'un içerisinde.
                Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

            """

    #################################### Kategorik Değişkenleri Oluşturacak Bölüm ######################################

    # Kategorik değişkenleri tip bilgisine göre yakalama
    cat_cols = [col for col in dataframe.columns if str(dataframe[col].dtypes) in ["category", "object", "bool"]]
    # cat_cols = [col for col in dataframe.columns if ((dataframe[col].dtypes == "object") | (dataframe[col].dtypes == "category") | (dataframe[col].dtypes == "bool"))]
    # Numerik değişken gibi gözüken ama kategorik değişken olanları yakalama
    num_but_cat = [col for col in dataframe.columns if
                   ((dataframe[col].nunique() < cat_th) and (str(dataframe[col].dtypes) in ["int64", "float64"]))]
    # Bir kategorik değişkenin 50 sınıfı olması onun muhtemelen bilgi taşımadığını gösterir.
    # Bunlara Kardinalitesi yüksek değişkenler denir. Kardinal değişkenlerdir.
    # Kategorik tipte olduğu halde kategorik olmayan değişkenleri yakalamak
    cat_but_car = [col for col in dataframe.columns if
                   dataframe[col].nunique() > car_th and str(dataframe[col].dtypes) in ["category", "object"]]
    # cat_but_car = [col for col in cat_cols if dataframe[col].nunique() > 20]
    # Kategorik değişkenleri cat_cols altına topluyoruz
    cat_cols = cat_cols + num_but_cat
    # cat_cols lardan cat_but_car ları çıkarma işlemi yani kategorik değiişkenlerden kardinal değişkenleri çıkardık.
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    ###################################### Numerik Değişkenleri Oluşturacak Bölüm ######################################

    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes in ["int64", "float64"]]
    num_cols = [col for col in num_cols if col not in cat_cols]

    ################################################ Raporlama Bölümü ##################################################

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    print("######################################################################################")
    return cat_cols, num_cols, cat_but_car

In [12]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)

Observations: 2919
Variables: 81
cat_cols: 43
num_cols: 38
cat_but_car: 0
num_but_cat: 0
######################################################################################


In [13]:
date = [col for col in df.columns if (("Yr" in col) or ("Year" in col))]
df[date] = df[date].apply(pd.to_datetime)

In [14]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)

Observations: 2919
Variables: 81
cat_cols: 43
num_cols: 34
cat_but_car: 0
num_but_cat: 0
######################################################################################


In [23]:
def target_summary_with_cat(dataframe, target, categorical_col):
    print(pd.DataFrame({"TARGET_MEAN": dataframe.groupby(categorical_col)[target].mean()}), end="\n\n\n")


In [24]:
for col in cat_cols:
    target_summary_with_cat(df, "SalePrice", col)

            TARGET_MEAN
MSZoning               
C (all)    74528.000000
FV        214014.061538
RH        131558.375000
RL        191004.994787
RM        126316.830275


          TARGET_MEAN
Street               
Grvl    130190.500000
Pave    181130.538514


         TARGET_MEAN
Alley               
Grvl   122219.080000
Pave   168000.585366


            TARGET_MEAN
LotShape               
IR1       206101.665289
IR2       239833.365854
IR3       216036.500000
Reg       164754.818378


               TARGET_MEAN
LandContour               
Bnk          143104.079365
HLS          231533.940000
Low          203661.111111
Lvl          180183.746758


            TARGET_MEAN
Utilities              
AllPub     180950.95682
NoSeWa     137500.00000


             TARGET_MEAN
LotConfig               
Corner     181623.425856
CulDSac    223854.617021
FR2        177934.574468
FR3        208475.000000
Inside     176938.047529


             TARGET_MEAN
LandSlope               
Gtl        179956.7

In [25]:
def outlier_thresholds(dataframe, col_name, q1=0.01, q3=0.99):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [26]:
def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False


In [27]:
for col in num_cols:
    print(col, check_outlier(df, col))

Id False
MSSubClass False
LotFrontage True
LotArea True
OverallQual False
OverallCond False
MasVnrArea False
BsmtFinSF1 True
BsmtFinSF2 False
BsmtUnfSF False
TotalBsmtSF True
1stFlrSF True
2ndFlrSF False
LowQualFinSF True
GrLivArea False
BsmtFullBath False
BsmtHalfBath False
FullBath False
HalfBath False
BedroomAbvGr False
KitchenAbvGr False
TotRmsAbvGrd False
Fireplaces False
GarageCars False
GarageArea False
WoodDeckSF True
OpenPorchSF True
EnclosedPorch True
3SsnPorch True
ScreenPorch False
PoolArea True
MiscVal True
MoSold False
SalePrice False


In [28]:
def missing_values_table(dataframe, na_name=False):
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]

    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df, end="\n")

    if na_name:
        return na_columns

In [29]:
missing_values_table(df)

              n_miss  ratio
PoolQC          2909  99.66
MiscFeature     2814  96.40
Alley           2721  93.22
Fence           2348  80.44
SalePrice       1459  49.98
FireplaceQu     1420  48.65
LotFrontage      486  16.65
GarageCond       159   5.45
GarageYrBlt      159   5.45
GarageFinish     159   5.45
GarageQual       159   5.45
GarageType       157   5.38
BsmtExposure      82   2.81
BsmtCond          82   2.81
BsmtQual          81   2.77
BsmtFinType2      80   2.74
BsmtFinType1      79   2.71
MasVnrType        24   0.82
MasVnrArea        23   0.79
MSZoning           4   0.14
Functional         2   0.07
BsmtHalfBath       2   0.07
BsmtFullBath       2   0.07
Utilities          2   0.07
KitchenQual        1   0.03
TotalBsmtSF        1   0.03
BsmtUnfSF          1   0.03
GarageCars         1   0.03
GarageArea         1   0.03
BsmtFinSF2         1   0.03
BsmtFinSF1         1   0.03
Exterior2nd        1   0.03
Exterior1st        1   0.03
SaleType           1   0.03
Electrical         1

In [30]:
## Aykırı Değer

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


In [31]:
for col in num_cols:
    replace_with_thresholds(df, col)

In [32]:
for col in num_cols:
    print(col, check_outlier(df, col))

Id False
MSSubClass False
LotFrontage False
LotArea False
OverallQual False
OverallCond False
MasVnrArea False
BsmtFinSF1 False
BsmtFinSF2 False
BsmtUnfSF False
TotalBsmtSF False
1stFlrSF False
2ndFlrSF False
LowQualFinSF False
GrLivArea False
BsmtFullBath False
BsmtHalfBath False
FullBath False
HalfBath False
BedroomAbvGr False
KitchenAbvGr False
TotRmsAbvGrd False
Fireplaces False
GarageCars False
GarageArea False
WoodDeckSF False
OpenPorchSF False
EnclosedPorch False
3SsnPorch False
ScreenPorch False
PoolArea False
MiscVal False
MoSold False
SalePrice False


In [33]:
 ## Eksik Değer

missing_values_table(df)

              n_miss  ratio
PoolQC          2909  99.66
MiscFeature     2814  96.40
Alley           2721  93.22
Fence           2348  80.44
SalePrice       1459  49.98
FireplaceQu     1420  48.65
LotFrontage      486  16.65
GarageCond       159   5.45
GarageYrBlt      159   5.45
GarageFinish     159   5.45
GarageQual       159   5.45
GarageType       157   5.38
BsmtExposure      82   2.81
BsmtCond          82   2.81
BsmtQual          81   2.77
BsmtFinType2      80   2.74
BsmtFinType1      79   2.71
MasVnrType        24   0.82
MasVnrArea        23   0.79
MSZoning           4   0.14
Functional         2   0.07
BsmtHalfBath       2   0.07
BsmtFullBath       2   0.07
Utilities          2   0.07
KitchenQual        1   0.03
TotalBsmtSF        1   0.03
BsmtUnfSF          1   0.03
GarageCars         1   0.03
GarageArea         1   0.03
BsmtFinSF2         1   0.03
BsmtFinSF1         1   0.03
Exterior2nd        1   0.03
Exterior1st        1   0.03
SaleType           1   0.03
Electrical         1

In [34]:
df = df.drop(["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu", "LotFrontage"], axis=1)
df.shape # (2919, 75)

(2919, 75)

In [35]:
missing_values_table(df)

              n_miss  ratio
SalePrice       1459  49.98
GarageCond       159   5.45
GarageQual       159   5.45
GarageFinish     159   5.45
GarageYrBlt      159   5.45
GarageType       157   5.38
BsmtCond          82   2.81
BsmtExposure      82   2.81
BsmtQual          81   2.77
BsmtFinType2      80   2.74
BsmtFinType1      79   2.71
MasVnrType        24   0.82
MasVnrArea        23   0.79
MSZoning           4   0.14
Utilities          2   0.07
BsmtFullBath       2   0.07
BsmtHalfBath       2   0.07
Functional         2   0.07
Electrical         1   0.03
KitchenQual        1   0.03
BsmtUnfSF          1   0.03
BsmtFinSF2         1   0.03
BsmtFinSF1         1   0.03
GarageCars         1   0.03
GarageArea         1   0.03
Exterior2nd        1   0.03
Exterior1st        1   0.03
SaleType           1   0.03
TotalBsmtSF        1   0.03


In [36]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)

Observations: 2919
Variables: 75
cat_cols: 38
num_cols: 33
cat_but_car: 0
num_but_cat: 0
######################################################################################


In [37]:
df.isnull().sum().sort_values(ascending=False).head(30)

SalePrice       1459
GarageCond       159
GarageYrBlt      159
GarageFinish     159
GarageQual       159
GarageType       157
BsmtExposure      82
BsmtCond          82
BsmtQual          81
BsmtFinType2      80
BsmtFinType1      79
MasVnrType        24
MasVnrArea        23
MSZoning           4
Functional         2
Utilities          2
BsmtFullBath       2
BsmtHalfBath       2
GarageArea         1
BsmtFinSF1         1
BsmtUnfSF          1
SaleType           1
TotalBsmtSF        1
KitchenQual        1
BsmtFinSF2         1
Exterior2nd        1
Exterior1st        1
Electrical         1
GarageCars         1
BedroomAbvGr       0
dtype: int64

In [38]:
df["SalePrice"] = df["SalePrice"].fillna(0)

In [39]:
df = df.dropna()

In [40]:
df.shape 

(2657, 75)

In [41]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

In [42]:
def rare_analyser(dataframe, target, cat_cols):
    for col in cat_cols:
        print(col, len(dataframe[col].value_counts()))
        print(pd.DataFrame({"COUNT": dataframe[col].value_counts(),
                            "RATIO": dataframe[col].value_counts() / len(dataframe),
                            "TARGET_MEAN": dataframe.groupby(col)[target].mean()}), end="\n\n\n")

In [43]:
def cat_summary(dataframe, col_name, plot=False):
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": (dataframe[col_name].value_counts() / len(dataframe)) * 100}))
    print("################################################################################")
    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.title(col_name)
        plt.show(block=True)

In [44]:
for col in cat_cols:
    cat_summary(df, col)

         MSZoning      Ratio
RL           2097  78.923598
RM            396  14.904027
FV            131   4.930373
RH             20   0.752729
C (all)        13   0.489274
################################################################################
      Street      Ratio
Pave    2648  99.661272
Grvl       9   0.338728
################################################################################
     LotShape      Ratio
Reg      1649  62.062476
IR1       920  34.625518
IR2        73   2.747460
IR3        15   0.564546
################################################################################
     LandContour      Ratio
Lvl         2390  89.951073
HLS          115   4.328190
Bnk           99   3.726007
Low           53   1.994731
################################################################################
        Utilities      Ratio
AllPub       2656  99.962364
NoSeWa          1   0.037636
##############################################################################

In [45]:
def rare_encoder(dataframe, rare_perc):
    temp_df = dataframe.copy()

    rare_columns = [col for col in temp_df.columns if temp_df[col].dtypes == "O"
                    and (temp_df[col].value_counts() / len(temp_df) < rare_perc).any(axis=None)]
    # Kategorik değişkenein frekanslarını aldık ve bunları toplam gözlem sayısına böldük ve bu oran belirlediğimiz
    # orandan(rare_perc) herhangi bir tanesi küçük ve tipide object ise bunları rare_columns olarak alıyoruz.

    for var in rare_columns:
        tmp = temp_df[var].value_counts() / len(temp_df) # temp_df içindeki ilgili rare değişkeni için bir sınıf oranları hesaplanır.
        rare_labels = tmp[tmp < rare_perc].index
        temp_df[var] = np.where(temp_df[var].isin(rare_labels), "Rare", temp_df[var])
        # temp_df de rare_labels gördüğün yerlere "Rare" yaz değilse olduğu gibi kalsın.
    return temp_df


In [46]:
# %1'den küçükleri Rare aldık.
df = rare_encoder(df, 0.01)

In [47]:
for col in cat_cols:
    cat_summary(df, col)

      MSZoning      Ratio
RL        2097  78.923598
RM         396  14.904027
FV         131   4.930373
Rare        33   1.242002
################################################################################
      Street      Ratio
Pave    2648  99.661272
Rare       9   0.338728
################################################################################
      LotShape      Ratio
Reg       1649  62.062476
IR1        920  34.625518
IR2         73   2.747460
Rare        15   0.564546
################################################################################
     LandContour      Ratio
Lvl         2390  89.951073
HLS          115   4.328190
Bnk           99   3.726007
Low           53   1.994731
################################################################################
        Utilities      Ratio
AllPub       2656  99.962364
Rare            1   0.037636
################################################################################
         LotConfig      Ratio
Inside

In [48]:
df["year"] = pd.qcut(df["YearBuilt"], 2, labels=["eski", "yeni"]).astype("object")
df["size"] = pd.qcut(df["LotArea"], 2, labels= ["kucuk", "buyuk"]).astype("object")

In [49]:
df.loc[(df["year"] == "eski") & (df["size"] == "kucuk") & (df["OverallCond"] <= 5), 'Segment'] = "C_sinifi"
df.loc[(df["year"] == "eski") & (df["size"] == "kucuk") & (df["OverallCond"] > 5), 'Segment'] = "B_sinifi"
df.loc[(df["year"] == "eski") & (df["size"] == "buyuk") & (df["OverallCond"] <= 5), 'Segment'] = "B_sinifi"
df.loc[(df["year"] == "eski") & (df["size"] == "buyuk") & (df["OverallCond"] > 5), 'Segment'] = "A_sinifi"
df.loc[(df["year"] == "yeni") & (df["size"] == "kucuk") & (df["OverallCond"] <= 5), 'Segment'] = "C_sinifi"
df.loc[(df["year"] == "yeni") & (df["size"] == "kucuk") & (df["OverallCond"] > 5), 'Segment'] = "A_sinifi"
df.loc[(df["year"] == "yeni") & (df["size"] == "buyuk") & (df["OverallCond"] <= 5), 'Segment'] = "B_sinifi"
df.loc[(df["year"] == "yeni") & (df["size"] == "buyuk") & (df["OverallCond"] > 5), 'Segment'] = "A_sinifi"

In [50]:
df["Segment"].value_counts()

B_sinifi    1345
C_sinifi     786
A_sinifi     526
Name: Segment, dtype: int64

In [51]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)

Observations: 2657
Variables: 78
cat_cols: 41
num_cols: 33
cat_but_car: 0
num_but_cat: 0
######################################################################################


In [52]:
def label_encoder(dataframe, binary_col):
    labelencoder = LabelEncoder()
    dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col])
    return dataframe


In [53]:
binary_cols = [col for col in df.columns if df[col].dtypes == "O" and df[col].nunique() == 2]
binary_cols

['Street',
 'Utilities',
 'Condition2',
 'RoofMatl',
 'Heating',
 'CentralAir',
 'year',
 'size']

In [54]:
for col in binary_cols:
    df = label_encoder(df, col)

In [55]:
df.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,year,size,Segment
0,1.0,60,RL,8450.0,0,Reg,Lvl,0,Inside,Gtl,CollgCr,Norm,0,1Fam,2Story,7.0,5,1970-01-01 00:00:00.000002003,1970-01-01 00:00:00.000002003,Gable,0,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,0,Ex,1,SBrkr,856.0,854.0,0.0,1710.0,1.0,0.0,2,1.0,3,1.0,Gd,8.0,Typ,0,Attchd,1970-01-01 00:00:00.000002003,RFn,2.0,548.0,TA,TA,Y,0.0,61.0,0,0,0.0,0,0.0,2.0,1970-01-01 00:00:00.000002008,WD,Normal,208500.0,1,1,C_sinifi
1,2.0,20,RL,9600.0,0,Reg,Lvl,0,FR2,Gtl,Rare,Feedr,0,1Fam,1Story,6.0,8,1970-01-01 00:00:00.000001976,1970-01-01 00:00:00.000001976,Gable,0,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,0,Ex,1,SBrkr,1262.0,0.0,0.0,1262.0,0.0,1.0,2,0.0,3,1.0,TA,6.0,Typ,1,Attchd,1970-01-01 00:00:00.000001976,RFn,2.0,460.0,TA,TA,Y,298.0,0.0,0,0,0.0,0,0.0,5.0,1970-01-01 00:00:00.000002007,WD,Normal,181500.0,0,0,A_sinifi
2,3.0,60,RL,11250.0,0,IR1,Lvl,0,Inside,Gtl,CollgCr,Norm,0,1Fam,2Story,7.0,5,1970-01-01 00:00:00.000002001,1970-01-01 00:00:00.000002002,Gable,0,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,0,Ex,1,SBrkr,920.0,866.0,0.0,1786.0,1.0,0.0,2,1.0,3,1.0,Gd,6.0,Typ,1,Attchd,1970-01-01 00:00:00.000002001,RFn,2.0,608.0,TA,TA,Y,0.0,42.0,0,0,0.0,0,0.0,9.0,1970-01-01 00:00:00.000002008,WD,Normal,223500.0,1,0,B_sinifi
3,4.0,70,RL,9550.0,0,IR1,Lvl,0,Corner,Gtl,Crawfor,Norm,0,1Fam,2Story,7.0,5,1970-01-01 00:00:00.000001915,1970-01-01 00:00:00.000001970,Gable,0,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,0,Gd,1,SBrkr,961.0,756.0,0.0,1717.0,1.0,0.0,1,0.0,3,1.0,Gd,7.0,Typ,1,Detchd,1970-01-01 00:00:00.000001998,Unf,3.0,642.0,TA,TA,Y,0.0,35.0,272,0,0.0,0,0.0,2.0,1970-01-01 00:00:00.000002006,WD,Abnorml,140000.0,0,1,C_sinifi
4,5.0,60,RL,14260.0,0,IR1,Lvl,0,FR2,Gtl,NoRidge,Norm,0,1Fam,2Story,8.0,5,1970-01-01 00:00:00.000002000,1970-01-01 00:00:00.000002000,Gable,0,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,0,Ex,1,SBrkr,1145.0,1053.0,0.0,2198.0,1.0,0.0,2,1.0,4,1.0,Gd,9.0,Typ,1,Attchd,1970-01-01 00:00:00.000002000,RFn,3.0,836.0,TA,TA,Y,192.0,84.0,0,0,0.0,0,0.0,12.0,1970-01-01 00:00:00.000002008,WD,Normal,250000.0,1,0,B_sinifi


In [56]:
def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe


In [57]:
df = one_hot_encoder(df, cat_cols, drop_first=True)

In [58]:
df.head()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,MSZoning_RL,MSZoning_RM,MSZoning_Rare,Street_1,LotShape_IR2,LotShape_Rare,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_1,LotConfig_CulDSac,LotConfig_FR2,LotConfig_Inside,LotConfig_Rare,LandSlope_Mod,LandSlope_Rare,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_Rare,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Condition1_Feedr,Condition1_Norm,Condition1_PosN,Condition1_RRAn,Condition1_Rare,Condition2_1,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1Story,HouseStyle_2Story,HouseStyle_Rare,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Hip,RoofStyle_Rare,RoofMatl_1,Exterior1st_BrkFace,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Rare,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_BrkFace,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_Rare,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_None,MasVnrType_Rare,MasVnrType_Stone,ExterQual_Gd,ExterQual_Rare,ExterQual_TA,ExterCond_Gd,ExterCond_Rare,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Rare,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Gd,BsmtCond_Rare,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_1,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Rare,HeatingQC_TA,CentralAir_1,Electrical_FuseF,Electrical_Rare,Electrical_SBrkr,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Min2,Functional_Rare,Functional_Typ,GarageType_Basment,GarageType_BuiltIn,GarageType_Detchd,GarageType_Rare,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Rare,GarageQual_TA,GarageCond_Rare,GarageCond_TA,PavedDrive_P,PavedDrive_Y,SaleType_New,SaleType_Rare,SaleType_WD,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_Rare,year_1,size_1,Segment_B_sinifi,Segment_C_sinifi
0,1.0,60,8450.0,7.0,5,1970-01-01 00:00:00.000002003,1970-01-01 00:00:00.000002003,196.0,706.0,0.0,150.0,856.0,856.0,854.0,0.0,1710.0,1.0,0.0,2,1.0,3,1.0,8.0,0,1970-01-01 00:00:00.000002003,2.0,548.0,0.0,61.0,0,0,0.0,0,0.0,2.0,1970-01-01 00:00:00.000002008,208500.0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,1,0,1
1,2.0,20,9600.0,6.0,8,1970-01-01 00:00:00.000001976,1970-01-01 00:00:00.000001976,0.0,978.0,0.0,284.0,1262.0,1262.0,0.0,0.0,1262.0,0.0,1.0,2,0.0,3,1.0,6.0,1,1970-01-01 00:00:00.000001976,2.0,460.0,298.0,0.0,0,0,0.0,0,0.0,5.0,1970-01-01 00:00:00.000002007,181500.0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0
2,3.0,60,11250.0,7.0,5,1970-01-01 00:00:00.000002001,1970-01-01 00:00:00.000002002,162.0,486.0,0.0

In [59]:
df.shape

(2657, 182)

In [60]:
test_df = df.loc[df["SalePrice"] == 0]
test_df.shape

(1319, 182)

In [61]:
train_df = df.loc[df["SalePrice"] != 0]
train_df.shape

(1338, 182)

In [62]:
X = train_df.drop(["Id", "SalePrice", 'YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold'], axis=1)
X.shape

(1338, 176)

In [63]:
y =train_df["SalePrice"]
y.shape

(1338,)

In [64]:
rf_model = RandomForestRegressor(random_state=17)

In [65]:
rf_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 17,
 'verbose': 0,
 'warm_start': False}

In [66]:
cv_results = cross_validate(rf_model, X, y, cv=5, scoring=('r2', 'neg_mean_squared_error'))

In [67]:
cv_results

{'fit_time': array([1.50418329, 1.47111535, 1.55217338, 1.47502446, 1.46013594]),
 'score_time': array([0.01519823, 0.01000023, 0.01100278, 0.01000023, 0.01000047]),
 'test_r2': array([0.86208228, 0.82238459, 0.87807994, 0.88790388, 0.81161826]),
 'test_neg_mean_squared_error': array([-7.47107586e+08, -1.16376099e+09, -9.10005044e+08, -6.63790019e+08,
        -1.07337337e+09])}

In [68]:
# RMSE
np.mean((np.sqrt(-cv_results["test_neg_mean_squared_error"])))

30027.999999254225

In [69]:
# R2
cv_results["test_r2"].mean()

0.8524137872058594

In [70]:
rf_params = {"max_depth": [5, 8, None],
             "max_features": [3, 5, 7, "auto"],
             "min_samples_split": [2, 5, 8, 15, 20],
             "n_estimators": [100, 200, 500]}

In [71]:
# en iyi parametreleri bulma işlemi.
rf_best_grid = GridSearchCV(rf_model,
                            rf_params,
                            cv=5,
                            n_jobs=-1,
                            verbose=True).fit(X, y)

Fitting 5 folds for each of 180 candidates, totalling 900 fits


In [72]:
rf_best_grid.best_params_

{'max_depth': None,
 'max_features': 'auto',
 'min_samples_split': 8,
 'n_estimators': 100}

In [73]:
rf_final = rf_model.set_params(**rf_best_grid.best_params_, random_state=17).fit(X, y)

In [74]:
rf_final

RandomForestRegressor(max_features='auto', min_samples_split=8, random_state=17)

In [75]:
cv_results = cross_validate(rf_model, X, y, cv=5, scoring=('r2', 'neg_mean_squared_error'))

In [76]:
cv_results

{'fit_time': array([1.25021529, 1.31215501, 1.28881884, 1.20187211, 1.18513608]),
 'score_time': array([0.01000047, 0.01000023, 0.01000094, 0.00995779, 0.0090003 ]),
 'test_r2': array([0.86326982, 0.83825805, 0.87606353, 0.88888452, 0.80986284]),
 'test_neg_mean_squared_error': array([-7.40674596e+08, -1.05975583e+09, -9.25055414e+08, -6.57983042e+08,
        -1.08337550e+09])}

In [77]:
# RMSE
np.mean((np.sqrt(-cv_results["test_neg_mean_squared_error"])))

29749.9604887596

In [78]:
# R2
cv_results["test_r2"].mean()

0.8552677517983576